In [1]:
import numpy as np 
import pandas as pd

In [90]:
data = np.loadtxt("b_should_be_easy.in",dtype=int)
rows,columns,n_vehicles,n_rides,bonus,steps = data[0,:]
actual_data = data[1:,:]
#row_start, column_start, row_end, column_end, earliest_start, latest finish
print(actual_data)

def distance(a,b,c,d):
    return abs(a-c) + abs(b-d)

distance(actual_data[0,0], actual_data[0,1], actual_data[0,2], actual_data[0,3])

all_distance = distance(actual_data[:,0], actual_data[:,1], actual_data[:,2], actual_data[:,3])
data_with_distances = np.hstack((actual_data, np.reshape(all_distance, (actual_data.shape[0],1))))
print(data_with_distances)
data_with_distances.shape

[[  395    43   296   955   244  3620]
 [  734   467   300   833 21919 23595]
 [  473   815    21   810 24428 24959]
 ..., 
 [  606   724   154   616 20196 21068]
 [  463   972   343   532 13263 14196]
 [   71   674    33   713  3467  3562]]
[[  395    43   296 ...,   244  3620  1011]
 [  734   467   300 ..., 21919 23595   800]
 [  473   815    21 ..., 24428 24959   457]
 ..., 
 [  606   724   154 ..., 20196 21068   560]
 [  463   972   343 ..., 13263 14196   560]
 [   71   674    33 ...,  3467  3562    77]]


(300, 7)

In [91]:
#last column is the latest start
data_with_latest_start = np.hstack((data_with_distances, np.reshape(data_with_distances[:,5]-data_with_distances[:,-1],(data_with_distances.shape[0],1))))
sorted_data = data_with_latest_start[data_with_latest_start[:,-1].argsort()]

#data sorted according to the latest start (last column)
print(sorted_data)


[[  518   656   201 ...,   712   479   233]
 [  662   421   314 ...,   656   372   284]
 [  623   425   588 ...,  1239   516   723]
 ..., 
 [  505   566   462 ..., 24865   323 24542]
 [  379   411   648 ..., 24952   270 24682]
 [  528   267   653 ..., 24925   177 24748]]


In [94]:
submit = {}
for i in range(n_vehicles):
    submit['F%s'%i] = [1,i]
    #if i > 32:
    #    submit['F%s'%i] = [0]

In [95]:
def calc_points(bonus, rides_data, submission_dict):
    assert isinstance(rides_data,(list,np.ndarray))
    assert isinstance(submission_dict,dict)
    dist_points = 0
    bonus_points = 0
    
    for f in submission_dict.keys():
        f_rides = submission_dict[f]
        assert len(f_rides) == f_rides[0] + 1, 'First number in list for each vehicle must equal the # of rides, occured for vehicle %s'%f
        
        f_position = np.array([0,0])
        t = 0
        for ride_num in f_rides[1:]:
            get_bonus = False
            start_row,start_col,end_row,end_col,earliest_start,latest_finish,distance = rides_data[ride_num]
            print(rides_data[ride_num])
            dist_to_start = np.sum(np.abs(f_position - np.array([start_row,start_col])))
            t += dist_to_start
            
            if t < earliest_start:
                print('t less than earliest start %s, %s'%(t, earliest_start))
                bonus_points += bonus 
                get_bonus=True
            
            dist_points += distance
            t = max(t,earliest_start) + distance
            f_position = [end_row,end_col]
            
            if t > latest_finish:
                print('Warning, vehicle %s could not complete ride %s on time, finished at %s, should have finished at %s'%(f,ride_num,t,latest_finish))
                dist_points -= distance
                if get_bonus == True:
                    bonus_points -= bonus
            
    return dist_points + bonus_points
            
            
        
    print(bonus, rides_data, submission_dict)
calc_points(bonus, data_with_distances, submit)

[ 395   43  296  955  244 3620 1011]
[  734   467   300   833 21919 23595   800]
t less than earliest start 1201, 21919
[  473   815    21   810 24428 24959   457]
t less than earliest start 1288, 24428
[  210   150   387    12 12646 13491   315]
t less than earliest start 360, 12646
[ 314  563  141  419 2744 3381  317]
t less than earliest start 877, 2744
[  237   432   151   805 16506 17940   459]
t less than earliest start 669, 16506
[  264   491   445   767 14518 16078   457]
t less than earliest start 755, 14518
[ 219  215  154  449 2806 3698  299]
t less than earliest start 434, 2806
[ 508  681  180  701 2744 3179  348]
t less than earliest start 1189, 2744
[  460   585   320   715 23246 23518   270]
t less than earliest start 1045, 23246
[  732   600   101   985 15613 19024  1016]
t less than earliest start 1332, 15613
[  790   935   686   121 12617 14630   918]
t less than earliest start 1725, 12617
[  761    85   111   567 18517 21011  1132]
t less than earliest start 846, 185

55731

In [85]:
def submission_dict_to_file(submission_dict,output_file_name = 'out.txt'):
    assert isinstance(submission_dict,dict)
    file = ''
    for f in submission_dict.keys():
        f_rides = [str(int(x)) for x in submission_dict[f]]
        file += ' '.join(f_rides)
        file += '\n'
    text_file = open(output_file_name, "w")
    text_file.write(file)
    text_file.close()
submission_dict_to_file(submit,'b_out.txt')